# Re-log Failed Database Entries (Synchronous Version)

This notebook helps re-insert failed log entries into the Neon database using synchronous connections.

In [ ]:
import os
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Import database models
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
from utils.database import ChatLog, Base, get_db_engine

## Paste Failed Log Parameters Here

In [ ]:
# Paste your failed log parameters here
# IMPORTANT: Replace "datetime.datetime" with just "datetime" when pasting!

failed_logs = [
    # Example already added:
    (datetime(2025, 6, 12, 17, 56, 36, 71342), 'U61539d6d1bd2f05b77ea99c84332ce93', 'new', '🆕 新對話開始。\n請選擇功能或直接傳送語音訊息：', 'sync reply', False, None, datetime(2025, 6, 12, 17, 56, 36, 71346)),
    # Add your failed log here (remember to replace datetime.datetime with datetime):
    (datetime(2025, 6, 13, 4, 58, 45, 7749), 'Uc721d654582e24d84713fddd7839865f', '大約要半年才會穩定，但這是需要終身服藥控制的疾病', '您是否想表達：\n大約需要半年時間病情才會穩定下來，不過這是一種需要終身服藥來控制的疾病。\n\nประมาณครึ่งปีอาการถึงจะคงที่ อย่างไรก็ตาม นี่เป็นโรคที่ต้องกินยาตลอดชีวิตเพื่อควบคุม เข้าใจไหมครับ?', 'medchat', True, 'https://drive.google.com/file/d/1S1gmaC8_udqWupFzQ2QGpP2ESeFdzVfE/view', datetime(2025, 6, 13, 4, 58, 45, 7753)),
]

## Process Failed Logs

In [ ]:
# Parse the failed logs into structured data
parsed_logs = []

for log_params in failed_logs:
    if len(log_params) == 8:
        timestamp, user_id, message, reply, action_type, gemini_call, gemini_output_url, created_at = log_params
        parsed_logs.append({
            'timestamp': timestamp,
            'user_id': user_id,
            'message': message,
            'reply': reply,
            'action_type': action_type,
            'gemini_call': gemini_call,
            'gemini_output_url': gemini_output_url,
            'created_at': created_at
        })
    else:
        print(f"Warning: Log entry has {len(log_params)} parameters instead of 8")

print(f"Parsed {len(parsed_logs)} log entries")
for i, log in enumerate(parsed_logs):
    print(f"\nLog {i+1}:")
    print(f"  User ID: {log['user_id'][:20]}...")
    print(f"  Message: {log['message'][:50]}..." if len(log['message']) > 50 else f"  Message: {log['message']}")
    print(f"  Action: {log['action_type']}")
    print(f"  Timestamp: {log['timestamp']}")

## Check Database Status

In [ ]:
def check_database_status():
    """Check current database status and sequences"""
    engine = get_db_engine()
    
    with engine.connect() as conn:
        # Check current max ID and sequence
        result = conn.execute(text("""
            SELECT 
                (SELECT MAX(id) FROM chat_logs) as max_id,
                (SELECT last_value FROM chat_logs_id_seq) as seq_value,
                (SELECT COUNT(*) FROM chat_logs) as total_count
        """))
        row = result.fetchone()
        print(f"Database Status:")
        print(f"  Total chat logs: {row.total_count}")
        print(f"  Max ID: {row.max_id}")
        print(f"  Current sequence: {row.seq_value}")
        
    engine.dispose()
    return row.max_id, row.seq_value

# Check current status
max_id, seq_value = check_database_status()

## Fix Sequence if Needed

In [ ]:
def fix_sequence_if_needed():
    """Fix sequence if it's out of sync"""
    engine = get_db_engine()
    
    with engine.begin() as conn:
        # Get current max ID
        result = conn.execute(text("SELECT COALESCE(MAX(id), 0) as max_id FROM chat_logs"))
        max_id = result.scalar()
        
        # Reset sequence to max_id + 1
        new_seq = max_id + 1
        conn.execute(text(f"ALTER SEQUENCE chat_logs_id_seq RESTART WITH {new_seq}"))
        print(f"✅ Sequence reset to: {new_seq}")
    
    engine.dispose()

# Fix sequence
fix_sequence_if_needed()

## Re-insert Failed Logs

In [ ]:
def reinsert_logs(logs):
    """Re-insert failed logs into the database"""
    engine = get_db_engine()
    Session = sessionmaker(bind=engine)
    
    success_count = 0
    failed_count = 0
    
    for log_data in logs:
        session = Session()
        try:
            # Create new ChatLog object (without specifying ID)
            chat_log = ChatLog(
                timestamp=log_data['timestamp'],
                user_id=log_data['user_id'],
                message=log_data['message'],
                reply=log_data['reply'][:1000] if log_data['reply'] else None,
                action_type=log_data['action_type'],
                gemini_call=log_data['gemini_call'],
                gemini_output_url=log_data['gemini_output_url'],
                created_at=log_data['created_at']
            )
            
            session.add(chat_log)
            session.commit()
            
            print(f"✅ Successfully re-inserted log for user {log_data['user_id'][:20]}... (ID: {chat_log.id})")
            success_count += 1
            
        except Exception as e:
            print(f"❌ Failed to re-insert log: {e}")
            session.rollback()
            failed_count += 1
        finally:
            session.close()
    
    engine.dispose()
    
    print(f"\n📊 Summary:")
    print(f"  Successfully re-inserted: {success_count}")
    print(f"  Failed: {failed_count}")
    
    return success_count, failed_count

# Re-insert the logs
if parsed_logs:
    success, failed = reinsert_logs(parsed_logs)
else:
    print("No logs to re-insert. Please add failed log parameters in the cell above.")

## Alternative: Direct SQL Insert

In [ ]:
def direct_sql_insert(logs):
    """Insert logs using direct SQL (bypasses ORM)"""
    engine = get_db_engine()
    
    success_count = 0
    failed_count = 0
    
    with engine.begin() as conn:
        for log_data in logs:
            try:
                # Use INSERT with column names (let ID auto-generate)
                conn.execute(text("""
                    INSERT INTO chat_logs 
                    (timestamp, user_id, message, reply, action_type, gemini_call, gemini_output_url, created_at)
                    VALUES 
                    (:timestamp, :user_id, :message, :reply, :action_type, :gemini_call, :gemini_output_url, :created_at)
                """), {
                    'timestamp': log_data['timestamp'],
                    'user_id': log_data['user_id'],
                    'message': log_data['message'],
                    'reply': log_data['reply'][:1000] if log_data['reply'] else None,
                    'action_type': log_data['action_type'],
                    'gemini_call': log_data['gemini_call'],
                    'gemini_output_url': log_data['gemini_output_url'],
                    'created_at': log_data['created_at']
                })
                
                print(f"✅ Successfully inserted log via SQL for user {log_data['user_id'][:20]}...")
                success_count += 1
                
            except Exception as e:
                print(f"❌ Failed to insert via SQL: {e}")
                failed_count += 1
    
    engine.dispose()
    
    print(f"\n📊 SQL Insert Summary:")
    print(f"  Successfully inserted: {success_count}")
    print(f"  Failed: {failed_count}")
    
# Uncomment to use direct SQL insert if ORM method fails
# if parsed_logs:
#     direct_sql_insert(parsed_logs)

## Verify Results

In [ ]:
# Check database status after insertion
check_database_status()

# Check specific user's recent logs
if parsed_logs:
    user_id = parsed_logs[0]['user_id']
    engine = get_db_engine()
    
    with engine.connect() as conn:
        result = conn.execute(text("""
            SELECT id, timestamp, action_type, LEFT(message, 50) as message_preview
            FROM chat_logs
            WHERE user_id = :user_id
            ORDER BY timestamp DESC
            LIMIT 5
        """), {'user_id': user_id})
        
        print(f"\nRecent logs for user {user_id[:20]}...:")
        for row in result:
            print(f"  ID: {row.id}, Time: {row.timestamp}, Action: {row.action_type}")
            print(f"    Message: {row.message_preview}..." if row.message_preview and len(row.message_preview) >= 50 else f"    Message: {row.message_preview}")
    
    engine.dispose()